In [1]:
import os
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html
import util.util as util
import argparse
import os
from util import util
import torch
import models
import data
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
class Options:
    def __init__(self):
        self.CUT_mode = 'CUT'
        self.batch_size = 1
        self.checkpoints_dir = './checkpoints'
        self.crop_size = 256
        self.dataroot = '/home/jun/CUT/datasets/market2duke'
        self.dataset_mode = 'unaligned'
        self.direction = 'AtoB'
        self.display_winsize = 256
        self.easy_label = 'market2duke'
        self.epoch = '10'
        self.eval = False
        self.flip_equivariance = False
        self.gpu_ids = [0]
        self.init_gain = 0.02
        self.init_type = 'xavier'
        self.input_nc = 3
        self.isTrain = False
        self.lambda_GAN = 1.0
        self.lambda_NCE = 1.0
        self.load_size = 256
        self.max_dataset_size = float("inf")
        self.model = 'cut'
        self.n_layers_D = 3
        self.name = 'market2duke_segcut'
        self.nce_T = 0.07
        self.nce_idt = True
        self.nce_includes_all_negatives_from_minibatch = False
        self.nce_layers = '0,4,8,12,16'
        self.ndf = 64
        self.netD = 'basic'
        self.netF = 'mlp_sample'
        self.netF_nc = 256
        self.netG = 'resnet_9blocks'
        self.ngf = 64
        self.no_antialias = False
        self.no_antialias_up = False
        self.no_dropout = True
        self.no_flip = False
        self.normD = 'instance'
        self.normG = 'instance'
        self.num_patches = 256
        self.num_test = 50
        self.num_threads = 4
        self.output_nc = 3
        self.phase = 'test'
        self.pool_size = 0
        self.preprocess = 'resize_and_crop'
        self.random_scale_max = 3.0
        self.results_dir = './results/'
        self.serial_batches = False
        self.stylegan2_G_num_downsampling = 1
        self.suffix = ''
        self.verbose = False

opt = Options()

In [4]:
import os
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html
import util.util as util
import argparse
import os
from util import util
import torch
import models
import data
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
class Options:
    def __init__(self):
        self.CUT_mode = 'CUT'
        self.batch_size = 1
        self.checkpoints_dir = './checkpoints'
        self.crop_size = 256
        self.dataroot = '/home/jun/CUT/datasets/market2duke'
        self.dataset_mode = 'unaligned'
        self.direction = 'AtoB'
        self.display_winsize = 256
        self.easy_label = 'market2duke'
        self.epoch = '10'
        self.eval = False
        self.flip_equivariance = False
        self.gpu_ids = [0]
        self.init_gain = 0.02
        self.init_type = 'xavier'
        self.input_nc = 3
        self.isTrain = False
        self.lambda_GAN = 1.0
        self.lambda_NCE = 1.0
        self.load_size = 256
        self.max_dataset_size = float("inf")
        self.model = 'cut'
        self.n_layers_D = 3
        self.name = 'duke2market_spcut'
        self.nce_T = 0.07
        self.nce_idt = True
        self.nce_includes_all_negatives_from_minibatch = False
        self.nce_layers = '0,4,8,12,16'
        self.ndf = 64
        self.netD = 'basic'
        self.netF = 'mlp_sample'
        self.netF_nc = 256
        self.netG = 'resnet_9blocks'
        self.ngf = 64
        self.no_antialias = False
        self.no_antialias_up = False
        self.no_dropout = True
        self.no_flip = False
        self.normD = 'instance'
        self.normG = 'instance'
        self.num_patches = 256
        self.num_test = 50
        self.num_threads = 4
        self.output_nc = 3
        self.phase = 'test'
        self.pool_size = 0
        self.preprocess = 'resize_and_crop'
        self.random_scale_max = 3.0
        self.results_dir = './results/'
        self.serial_batches = False
        self.stylegan2_G_num_downsampling = 1
        self.suffix = ''
        self.verbose = False

opt = Options()
#dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
model = create_model(opt)      # create a model given opt.model and other options
model.load_networks('latest')
# create a webpage for viewing the results
#web_dir = os.path.join(opt.results_dir, opt.name, '{}_{}'.format(opt.phase, opt.epoch))  # define the website directory
#print('creating web directory', web_dir)

model [CUTModel] was created
loading the model from ./checkpoints/duke2market_spcut/latest_net_G.pth


In [2]:
# Function to convert tensor to image for displaying
def tensor_to_image(tensor):
    tensor = tensor.clone().detach().cpu()  # Clone the tensor to avoid modifying the original and move to CPU
    tensor = F.interpolate(tensor, size=[256,128], mode='bilinear', align_corners=False)
    tensor = tensor.squeeze(0)  # Remove batch dimension
    tensor = tensor * torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1) + torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1)  # De-normalize
    tensor = tensor.permute(1, 2, 0)  # Change from CxHxW to HxWxC
    tensor = tensor.numpy()  # Convert to numpy array
    tensor = (tensor * 255).astype(np.uint8)  # Convert to uint8
    return tensor

crop_size_w = 256
crop_size_h = 256
# Dataset loader
transform = transforms.Compose([
    transforms.Resize((crop_size_h, crop_size_w)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [3]:
# Market1501 dataset dir
raw_data_dir_market1501 = Path('/home/jun/ReID_Dataset/Market-1501-v15.09.15')
# Make dictionary of all image market1501 with keys of image_name and values of image_path
market_train_dic = {i.name:i for i in sorted(list(raw_data_dir_market1501.glob('bounding_box_train/*.jpg')))}
market_gallery_dic = {i.name:i for i in sorted(list(raw_data_dir_market1501.glob('bounding_box_test/*.jpg')))}
market_query_dic = {i.name:i for i in sorted(list(raw_data_dir_market1501.glob('query/*.jpg')))}

new_data_dir_market1501 = Path('/home/jun/ReID_Dataset/Market-1501-v15.09.15-CUT-copilot')
new_data_dir_market1501.mkdir(exist_ok=True)
new_market_train_dir = new_data_dir_market1501 / 'bounding_box_train'
new_market_train_dir.mkdir(exist_ok=True)
new_market_gallery_dir = new_data_dir_market1501 / 'bounding_box_test'
new_market_gallery_dir.mkdir(exist_ok=True)
new_market_query_dir = new_data_dir_market1501 / 'query'
new_market_query_dir.mkdir(exist_ok=True)


for image_name, image_path in market_train_dic.items():
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image)
    x = transformed_image.unsqueeze(0).cuda()
    data = {'A':x,'B':x,
            'A_paths': image_path,
            'B_paths': image_path}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    # Assuming `data` is the dictionary containing the tensors

    output_image = tensor_to_image(visuals['fake_B'])
    output_image = Image.fromarray(output_image)
    output_image_path = new_market_train_dir / image_name
    output_image.save(output_image_path)

for image_name, image_path in market_gallery_dic.items():
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image)
    x = transformed_image.unsqueeze(0).cuda()
    data = {'A':x,'B':x,
            'A_paths': image_path,
            'B_paths': image_path}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    # Assuming `data` is the dictionary containing the tensors

    output_image = tensor_to_image(visuals['fake_B'])
    output_image = Image.fromarray(output_image)
    output_image_path = new_market_gallery_dir / image_name
    output_image.save(output_image_path)
    
for image_name, image_path in market_query_dic.items():
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image)
    x = transformed_image.unsqueeze(0).cuda()
    data = {'A':x,'B':x,
            'A_paths': image_path,
            'B_paths': image_path}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    # Assuming `data` is the dictionary containing the tensors

    output_image = tensor_to_image(visuals['fake_B'])
    output_image = Image.fromarray(output_image)
    output_image_path = new_market_query_dir / image_name
    output_image.save(output_image_path)

In [82]:
import os
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html
import util.util as util
import argparse
import os
from util import util
import torch
import models
import data
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

class Options:
    def __init__(self):
        self.CUT_mode = 'CUT'
        self.batch_size = 1
        self.checkpoints_dir = './checkpoints'
        self.crop_size = 256
        self.dataroot = '/home/jun/CUT/datasets/duke2market'
        self.dataset_mode = 'unaligned'
        self.direction = 'AtoB'
        self.display_winsize = 256
        self.easy_label = 'duke2market'
        self.epoch = 'latest'
        self.eval = False
        self.flip_equivariance = False
        self.gpu_ids = [0]
        self.init_gain = 0.02
        self.init_type = 'xavier'
        self.input_nc = 3
        self.isTrain = False
        self.lambda_GAN = 1.0
        self.lambda_NCE = 1.0
        self.load_size = 256
        self.max_dataset_size = float("inf")
        self.model = 'cut'
        self.n_layers_D = 3
        self.name = 'duke2market'
        self.nce_T = 0.07
        self.nce_idt = True
        self.nce_includes_all_negatives_from_minibatch = False
        self.nce_layers = '0,4,8,12,16'
        self.ndf = 64
        self.netD = 'basic'
        self.netF = 'mlp_sample'
        self.netF_nc = 256
        self.netG = 'resnet_9blocks'
        self.ngf = 64
        self.no_antialias = False
        self.no_antialias_up = False
        self.no_dropout = True
        self.no_flip = False
        self.normD = 'instance'
        self.normG = 'instance'
        self.num_patches = 256
        self.num_test = 50
        self.num_threads = 4
        self.output_nc = 3
        self.phase = 'test'
        self.pool_size = 0
        self.preprocess = 'resize_and_crop'
        self.random_scale_max = 3.0
        self.results_dir = './results/'
        self.serial_batches = False
        self.stylegan2_G_num_downsampling = 1
        self.suffix = ''
        self.verbose = False

opt = Options()

model = create_model(opt)      # create a model given opt.model and other options
model.load_networks('latest')


# duke1501 dataset dir
raw_data_dir_duke = Path('/home/jun/ReID_Dataset/DukeMTMC-reID')
# Make dictionary of all image duke1501 with keys of image_name and values of image_path
duke_train_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('bounding_box_train/*.jpg')))}
duke_gallery_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('bounding_box_test/*.jpg')))}
duke_query_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('query/*.jpg')))}

new_data_dir_duke = Path('/home/jun/ReID_Dataset/DukeMTMC-reID-marketstyle-CUT')
new_data_dir_duke.mkdir(exist_ok=True)
new_duke_train_dir = new_data_dir_duke / 'bounding_box_train'
new_duke_train_dir.mkdir(exist_ok=True)
new_duke_gallery_dir = new_data_dir_duke / 'bounding_box_test'
new_duke_gallery_dir.mkdir(exist_ok=True)
new_duke_query_dir = new_data_dir_duke / 'query'
new_duke_query_dir.mkdir(exist_ok=True)


for image_name, image_path in duke_train_dic.items():
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image)
    x = transformed_image.unsqueeze(0).cuda()
    data = {'A':x,'B':x,
            'A_paths': image_path,
            'B_paths': image_path}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    # Assuming `data` is the dictionary containing the tensors

    output_image = tensor_to_image(visuals['fake_B'])
    output_image = Image.fromarray(output_image)
    output_image_path = new_duke_train_dir / image_name
    output_image.save(output_image_path)

for image_name, image_path in duke_gallery_dic.items():
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image)
    x = transformed_image.unsqueeze(0).cuda()
    data = {'A':x,'B':x,
            'A_paths': image_path,
            'B_paths': image_path}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    # Assuming `data` is the dictionary containing the tensors

    output_image = tensor_to_image(visuals['fake_B'])
    output_image = Image.fromarray(output_image)
    output_image_path = new_duke_gallery_dir / image_name
    output_image.save(output_image_path)
    
for image_name, image_path in duke_query_dic.items():
    image = Image.open(image_path).convert('RGB')
    transformed_image = transform(image)
    x = transformed_image.unsqueeze(0).cuda()
    data = {'A':x,'B':x,
            'A_paths': image_path,
            'B_paths': image_path}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    # Assuming `data` is the dictionary containing the tensors

    output_image = tensor_to_image(visuals['fake_B'])
    output_image = Image.fromarray(output_image)
    output_image_path = new_duke_query_dir / image_name
    output_image.save(output_image_path)

model [CUTModel] was created
loading the model from ./checkpoints/duke2market/latest_net_G.pth


: 

In [4]:
image_name, image_path = list(market_train_dic.items())[201]
image = Image.open(image_path).convert('RGB')
transformed_image = transform(image)
x = transformed_image.unsqueeze(0).cuda()
data = {'A':x,'B':x,
        'A_paths': image_path,
        'B_paths': image_path}
model.set_input(data)
model.test()
visuals = model.get_current_visuals()
# Assuming `data` is the dictionary containing the tensors

real_A_img = tensor_to_image(visuals['real_A'])
fake_B_img = tensor_to_image(visuals['fake_B'])
real_B_img = tensor_to_image(visuals['real_B'])

# Plot the images
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Real A")
plt.imshow(real_A_img)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Fake B")
plt.imshow(fake_B_img)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Real B")
plt.imshow(real_B_img)
plt.axis('off')

plt.show()

NameError: name 'market_train_dic' is not defined